In [0]:
env = 'dev'

In [0]:
# ——————————————————————————————
#  ENV SETUP
# ——————————————————————————————
#env = dbutils.widgets.get("pipeline.env")
print(f"Setting up security policies for environment: {env}")

catalog = "book_rec_catalog"
schema = f"{env}_silver"

spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

print(f"Using catalog: {catalog}")
print(f"Using schema: {schema}")


In [0]:
# ——————————————————————————————
#  CREATE SECURITY FUNCTIONS
# ——————————————————————————————

print("Creating security functions...")

# Jedna masking funkce pro User-ID
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.mask_user_id(user_id STRING)
RETURNS STRING
RETURN CASE 
    WHEN is_account_group_member('data_scientists') THEN user_id
    WHEN is_account_group_member('analysts') THEN CONCAT('USER_', SUBSTRING(user_id, 1, 4), '***')
    ELSE '*** REDACTED ***'
END;
""")

In [0]:
# ——————————————————————————————
#  CREATE ROW FILTERING
# ——————————————————————————————
# Jedna row filtering funkce pro geografické omezení
spark.sql(f"""
CREATE OR REPLACE FUNCTION {catalog}.{schema}.rf_approved_regions(state STRING)
RETURNS BOOLEAN
RETURN CASE 
    WHEN is_account_group_member('viewers') THEN state = 'USA'
    WHEN is_account_group_member('analysts') THEN state IN ('USA', 'Canada', 'UK')
    ELSE TRUE
END;
""")

In [0]:
# ——————————————————————————————
#  APPLY SECURITY POLICIES
# ——————————————————————————————

print("Applying security policies...")

# Column masking na User-ID
try:
    spark.sql(f"""
    ALTER TABLE {catalog}.{schema}.users_silver_batch
    ALTER COLUMN `User-ID`
    SET MASK {catalog}.{schema}.mask_user_id
    """)
    print("Applied column masking to users_silver_batch.User-ID")
except Exception as e:
    print(f"Error applying column masking: {str(e)}")

# Row filtering na state
try:
    spark.sql(f"""
    ALTER TABLE {catalog}.{schema}.users_silver_batch
    SET ROW FILTER {catalog}.{schema}.rf_approved_regions
    ON (state)
    """)
    print("Applied row filtering to users_silver_batch on state")
except Exception as e:
    print(f"Error applying row filtering: {str(e)}")
